# Importing Libraries

In [50]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [51]:
import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import sys
import time
import sklearn
import xgboost as xgb
import prophet
import plotly.express as px
import plotly.graph_objects as go


In [79]:
from sklearn.model_selection import ParameterGrid
from datasets import load_dataset

# Loading Data

- It consists of hourly electricity demand (in GW) of the Victoria state in Australia together with the temperature (in Celsius degrees).

In [52]:
dataset = load_dataset("rajistics/electricity_demand")

Using custom data configuration rajistics--electricity_demand-810cb6002959ca42
Found cached dataset parquet (/Users/alijanatiidr/.cache/huggingface/datasets/rajistics___parquet/rajistics--electricity_demand-810cb6002959ca42/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 292.65it/s]


In [53]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Demand', 'Temperature', '__index_level_0__'],
        num_rows: 1344
    })
})

In [54]:
dataset['train']

Dataset({
    features: ['Demand', 'Temperature', '__index_level_0__'],
    num_rows: 1344
})

In [55]:
# Build a dataframe with the data
df_electricity_demand = pd.DataFrame(dataset['train'])
df_electricity_demand.head()

,Demand,Temperature,__index_level_0__
0,3.794,18.05,2014-01-01 00:00:00
1,3.418,17.20,2014-01-01 01:00:00
2,3.152,16.45,2014-01-01 02:00:00
3,3.026,16.65,2014-01-01 03:00:00
4,3.022,16.40,2014-01-01 04:00:00


In [56]:
df_electricity_demand.dtypes

Demand                      float64
Temperature                 float64
__index_level_0__    datetime64[ns]
dtype: object

# Parsing Data

In [57]:
# Parse dates
df_electricity_demand['Date'] = df_electricity_demand['__index_level_0__']
df_electricity_demand = df_electricity_demand[['Date', 'Temperature', 'Demand']]
df_electricity_demand.head()

,Date,Temperature,Demand
0,2014-01-01 00:00:00,18.05,3.794
1,2014-01-01 01:00:00,17.20,3.418
2,2014-01-01 02:00:00,16.45,3.152
3,2014-01-01 03:00:00,16.65,3.026
4,2014-01-01 04:00:00,16.40,3.022


# Exploratory Data Analysis

In [58]:
# checking for missing values
df_electricity_demand.isnull().sum()

Date           0
Temperature    0
Demand         0
dtype: int64

In [59]:
# checking for duplicates
df_electricity_demand.duplicated().sum()

0

In [60]:
# checking shape of the data
df_electricity_demand.shape

(1344, 3)

In [61]:
# plotting the data by scattering the points
fig = px.scatter(df_electricity_demand, x='Temperature', y='Demand', title='Electricity Demand in function of Temperature')
fig.show()

In [62]:
# plotting temporal data with temperature colorscale
fig = px.scatter(df_electricity_demand, x='Date', y='Demand', color='Temperature', title='Electricity Demand in function of Time with Temperature colorscale')
fig.show()

# Modelling

## Prophet

In [63]:
# Processing data for Prophet
df_electricity_demand_prophet = df_electricity_demand.copy()
df_electricity_demand_prophet = df_electricity_demand_prophet.rename(columns={'Date': 'ds', 'Demand': 'y'})
df_electricity_demand_prophet.head()

,ds,Temperature,y
0,2014-01-01 00:00:00,18.05,3.794
1,2014-01-01 01:00:00,17.20,3.418
2,2014-01-01 02:00:00,16.45,3.152
3,2014-01-01 03:00:00,16.65,3.026
4,2014-01-01 04:00:00,16.40,3.022


In [64]:
# Splitting data into train, validation and test sets
train = df_electricity_demand_prophet[:int(0.7*(len(df_electricity_demand_prophet)))]
valid = df_electricity_demand_prophet[int(0.7*(len(df_electricity_demand_prophet))):int(0.9*(len(df_electricity_demand_prophet)))]
test = df_electricity_demand_prophet[int(0.9*(len(df_electricity_demand_prophet))):]

In [65]:
train

,ds,Temperature,y
0,2014-01-01 00:00:00,18.05,3.794
1,2014-01-01 01:00:00,17.20,3.418
2,2014-01-01 02:00:00,16.45,3.152
3,2014-01-01 03:00:00,16.65,3.026
4,2014-01-01 04:00:00,16.40,3.022
...,...,...,...
935,2014-02-08 23:00:00,25.40,5.462
936,2014-02-09 00:00:00,25.60,4.824
937,2014-02-09 01:00:00,28.15,4.444
938,2014-02-09 02:00:00,32.40,4.237


In [66]:
# building baseline model
baseline_model = prophet.Prophet()
baseline_model.fit(train)

# predicting on validation set
forecast = baseline_model.predict(valid)

# creating a dataframe to store the predictions and actual values
pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
pred['y'] = valid['y'].reset_index(drop=True)
pred.head()


20:59:23 - cmdstanpy - INFO - Chain [1] start processing
20:59:23 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper,y
0,2014-02-09 04:00:00,3.704650,2.766808,4.704684,4.164
1,2014-02-09 05:00:00,4.077657,3.092477,4.991477,4.275
2,2014-02-09 06:00:00,4.507502,3.533309,5.403762,4.545
3,2014-02-09 07:00:00,4.872280,3.916400,5.890492,5.033
4,2014-02-09 08:00:00,5.130112,4.156343,6.177545,5.594


In [67]:
# plotting the predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=pred['ds'], y=pred['yhat'], name='Predicted'))
fig.add_trace(go.Scatter(x=pred['ds'], y=pred['y'], name='Actual'))
fig.update_layout(title='Predicted vs Actual Electricity Demand', xaxis_title='Date', yaxis_title='Electricity Demand')
fig.show()

In [70]:
# compute mape of the baseline model
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape(pred['y'], pred['yhat'])


31.837610090973435

In [71]:
# building a model with temperature as a regressor
model = prophet.Prophet()
model.add_regressor('Temperature')
model.fit(train)

# predicting on validation set
forecast = model.predict(valid)

# creating a dataframe to store the predictions and actual values
pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
pred['y'] = valid['y'].reset_index(drop=True)
pred.head()

21:00:32 - cmdstanpy - INFO - Chain [1] start processing
21:00:33 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper,y
0,2014-02-09 04:00:00,5.047856,4.399645,5.688383,4.164
1,2014-02-09 05:00:00,5.389405,4.711935,5.985604,4.275
2,2014-02-09 06:00:00,5.723432,5.066485,6.352008,4.545
3,2014-02-09 07:00:00,6.041418,5.366765,6.667798,5.033
4,2014-02-09 08:00:00,6.147556,5.488414,6.806929,5.594


In [72]:
# plotting the predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=pred['ds'], y=pred['yhat'], name='Predicted'))
fig.add_trace(go.Scatter(x=pred['ds'], y=pred['y'], name='Actual'))
fig.update_layout(title='Predicted vs Actual Electricity Demand', xaxis_title='Date', yaxis_title='Electricity Demand')
fig.show()

In [73]:
# compute mape of the model with temperature as a regressor
mape(pred['y'], pred['yhat'])

7.18982239977248

In [80]:
# fine tuning previous model with grid search

# creating a dictionary of hyperparameters
params = {'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
            'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
            'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0],
            'seasonality_mode': ['additive', 'multiplicative'],
            'changepoint_range': [0.8, 0.9, 1.0]}
grid = ParameterGrid(params)

# creating a dataframe to store the results
results = pd.DataFrame(columns=['mape', 'params'])

# iterating over the grid
for g in grid:
    model = prophet.Prophet(**g).fit(train)  # Fit model with given params
    forecast = model.predict(valid)
    pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    pred['y'] = valid['y'].reset_index(drop=True)
    mape_ = mape(pred['y'], pred['yhat'])
    results = results.append({'mape': mape_, 'params': g}, ignore_index=True)

results

21:17:15 - cmdstanpy - INFO - Chain [1] start processing
21:17:15 - cmdstanpy - INFO - Chain [1] done processing
21:17:16 - cmdstanpy - INFO - Chain [1] start processing
21:17:16 - cmdstanpy - INFO - Chain [1] done processing
21:17:17 - cmdstanpy - INFO - Chain [1] start processing
21:17:17 - cmdstanpy - INFO - Chain [1] done processing
21:17:17 - cmdstanpy - INFO - Chain [1] start processing
21:17:17 - cmdstanpy - INFO - Chain [1] done processing
21:17:18 - cmdstanpy - INFO - Chain [1] start processing
21:17:18 - cmdstanpy - INFO - Chain [1] done processing
21:17:18 - cmdstanpy - INFO - Chain [1] start processing
21:17:18 - cmdstanpy - INFO - Chain [1] done processing
21:17:19 - cmdstanpy - INFO - Chain [1] start processing
21:17:19 - cmdstanpy - INFO - Chain [1] done processing
21:17:19 - cmdstanpy - INFO - Chain [1] start processing
21:17:19 - cmdstanpy - INFO - Chain [1] done processing
21:17:20 - cmdstanpy - INFO - Chain [1] start processing
21:17:20 - cmdstanpy - INFO - Chain [1]

,mape,params
0,23.369744,"{'changepoint_prior_scale': 0.001, 'changepoin..."
1,22.214336,"{'changepoint_prior_scale': 0.001, 'changepoin..."
2,22.205432,"{'changepoint_prior_scale': 0.001, 'changepoin..."
3,22.209130,"{'changepoint_prior_scale': 0.001, 'changepoin..."
4,22.053772,"{'changepoint_prior_scale': 0.001, 'changepoin..."
...,...,...
379,123.804970,"{'changepoint_prior_scale': 0.5, 'changepoint_..."
380,85.133431,"{'changepoint_prior_scale': 0.5, 'changepoint_..."
381,123.622413,"{'changepoint_prior_scale': 0.5, 'changepoint_..."
382,129.550008,"{'changepoint_prior_scale': 0.5, 'changepoint_..."


In [81]:
# get best parameters
results.loc[results['mape'].idxmin()]

mape                                              13.416972
params    {'changepoint_prior_scale': 0.01, 'changepoint...
Name: 134, dtype: object

- Best model for time series forecasting seems to be the baseline model with temperature as regressor.

# Generalization capability

In [82]:
# Training best model on train + validation sets and predicting on test set to get generalization error
best_model = prophet.Prophet()
best_model.add_regressor('Temperature')

# concatenating train and validation sets
train_valid = pd.concat([train, valid])

# fitting the model
best_model.fit(train_valid)

# predicting on test set
forecast = best_model.predict(test)

# creating a dataframe to store the predictions and actual values
pred = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
pred['y'] = test['y'].reset_index(drop=True)
pred.head()

21:27:40 - cmdstanpy - INFO - Chain [1] start processing
21:27:40 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper,y
0,2014-02-20 09:00:00,4.436174,3.847558,5.048489,4.741
1,2014-02-20 10:00:00,4.563369,3.921611,5.162189,4.846
2,2014-02-20 11:00:00,4.689396,4.106511,5.254185,4.864
3,2014-02-20 12:00:00,4.828630,4.211896,5.426114,4.868
4,2014-02-20 13:00:00,4.889169,4.280385,5.450415,4.836


In [83]:
# plotting the predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=pred['ds'], y=pred['yhat'], name='Predicted'))
fig.add_trace(go.Scatter(x=pred['ds'], y=pred['y'], name='Actual'))
fig.update_layout(title='Predicted vs Actual Electricity Demand', xaxis_title='Date', yaxis_title='Electricity Demand')
fig.show()

In [84]:
# compute mape to get generalization error
mape(pred['y'], pred['yhat'])

8.334987119863985

- Conlusion: The model is able to generalize well to the test set.